# Introduction : Imports et préparation du dataset

In [2]:
from gurobipy import *
import pandas as pd

In [3]:
# Import des poids
df_weights = pd.read_excel(
    "RATP.xlsx",
    usecols="C:I",
    skiprows=13,
    nrows=1,
) # type: ignore

# Imports des données pour chaque station
data = pd.read_excel(
    "RATP.xlsx",
    usecols="B:I",
    skiprows=2,
    nrows=10,
    index_col=0,
)

# Réécriture des poids dans un dictionnaire
df_weights.columns = data.columns
weights = df_weights.iloc[0].to_dict()

# Explication de type (1-m)

Pour chaque paire de stations, on effectue une analyse (1-m) pour vérifier l'existence d'une explication satisfaisante.

In [9]:
def analyse_1_m(data, weights,u, v, print_details=True):
    """
    Réalise une analyse (1-m) de la paire u, v

    Args:
        data (pd.DataFrame): Données des candidats
        weights (dict): Poids des critères
        u (str): Nom du candidat u
        v (str): Nom du candidat v
        print_details (bool): Indique si les détails doivent être affichés
    """
    
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (data.loc[u, k] - data.loc[v, k])  

    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]

    if print_details:
        print(f"Arguments Pour (Pros) : {pros}")
        print(f"Arguments Contre (Cons) : {cons}")

    # MODÉLISATION GUROBI
    m = Model("Explication_1_m")

    # Variables de décision UNIQUEMENT x
    # x[p, c] = 1 si l'argument Pro 'p' couvre l'argument Con 'c'
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

    m.update()

    # CONTRAINTES

    # A. Couverture complète : Chaque Con doit être couvert par exactement un Pro
    for c in cons:
        m.addConstr(quicksum(x[p, c] for p in pros) == 1, name=f"Cover_{c}")

    # B. Validité du Trade-off
    # Formule : Delta_p + Somme(Delta_c * x_pc) >= epsilon
    # Si p est utilisé, cela vérifie que le trade-off est positif.
    # Si p est INUTILISÉ (x_pc = 0 partout), cela devient Delta_p >= epsilon, ce qui est toujours VRAI.
    epsilon = 0.001 

    for p in pros:
        # Note: deltas[c] est négatif ici
        m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")

    # RÉSOLUTION
    m.params.outputflag = 0  
    m.optimize()

    # ANALYSE
    if print_details:
        print("-" * 30)
    if m.status == GRB.OPTIMAL:
        if print_details:
            print("Résultat : Une explication de type (1-m) existe !\n")
        
        # On itère sur les Pros et on vérifie s'ils ont des Cons assignés
        for p in pros:
            covered_cons = [c for c in cons if x[p, c].x > 0.5]
            
            # Si la liste n'est pas vide, c'est que ce Pro fait partie de l'explication
            if covered_cons:
                net_value = deltas[p] + sum(deltas[c] for c in covered_cons)
                if print_details:
                    print(f"Groupe mené par {p} (+{deltas[p]}) :")
                    print(f"  -> Couvre : {covered_cons} (Poids cumulé Cons: {sum(deltas[c] for c in covered_cons)})")
                    print(f"  -> Bilan net : +{net_value}")
                
        return True

    elif m.status == GRB.INFEASIBLE:
        if print_details:
            print("Résultat : INFEASIBLE (Certificat de non-existence).")
        
        return False
    
resultats = {}
for u in data.index:
    for v in data.index:
        if u == v:
            continue
        
        print(f"\nAnalyse (1-m) pour la paire ({u} > {v}) :")
        res = analyse_1_m(data, weights, u, v)
        
        resultats[(u, v)] = res
            


Analyse (1-m) pour la paire (Odéon (Ligne 4) > Place d'Italie (Lign 6)) :
Arguments Pour (Pros) : ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Arguments Contre (Cons) : ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
------------------------------
Résultat : Une explication de type (1-m) existe !

Groupe mené par peak-entering-passengers/h (+85.99376545200472) :
  -> Couvre : ['off-peak-entering-passengers/h'] (Poids cumulé Cons: -75.24454477050415)
  -> Bilan net : +10.749220681500574
Groupe mené par off-peak-passing-passengers/h (+96.7429861335053) :
  -> Couvre : ['Station degradation level ([0,20]  scale)'] (Poids cumulé Cons: -94.0556809631303)
  -> Bilan net : +2.687305170374998
Groupe mené par strategic priority [0,10] (+128.9906481780071) :
  -> Couvre : ['connectivity index [0,100]'] (Poids cumulé Cons: -112.86681715575621)
  -> Bilan net : +16.123831022250883

Analys

# Explication de type (m-1)

Pour chaque paire de station, on effectue une analyse (1-m) pour vérifier l'existence d'une explication suffisante.

In [11]:
def analyse_m_1(data, weights,u, v, print_details=True):
    """
    Réalise une analyse (m-1) de la paire u, v

    Args:
        data (pd.DataFrame): Données des candidats
        weights (dict): Poids des critères
        u (str): Nom du candidat u
        v (str): Nom du candidat v
        print_details (bool): Indique si les détails doivent être affichés
    """
    
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (data.loc[u, k] - data.loc[v, k])  

    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]

    if print_details:
        print(f"Arguments Pour (Pros) : {pros}")
        print(f"Arguments Contre (Cons) : {cons}")

    # MODÉLISATION GUROBI
    m = Model("Explication_m_1")

    # Variables de décision UNIQUEMENT x
    # x[p, c] = 1 si l'argument Pro 'p' est couvert par l'argument Con 'c'
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

    m.update()

    # CONTRAINTES

    # A. Couverture complète : Chaque Pro doit être couvert par exactement un Con
    for p in pros:
        m.addConstr(quicksum(x[p, c] for c in cons) == 1, name=f"Cover_{p}")

    # B. Validité du Trade-off
    # Formule : Delta_c + Somme(Delta_p * x_pc) >= epsilon
    # Si c est utilisé, cela vérifie que le trade-off est négatif suffisant.
    epsilon = 0.001 

    for c in cons:
        # Note: deltas[c] est négatif ici
        m.addConstr(deltas[c] + quicksum(deltas[p] * x[p, c] for p in pros) >= epsilon, name=f"Valid_{c}")

    # RÉSOLUTION
    m.params.outputflag = 0  
    m.optimize()

    # ANALYSE
    if print_details:
        print("-" * 30)
    if m.status == GRB.OPTIMAL:
        if print_details:
            print("Résultat : Une explication de type (m-1) existe !\n")
        
        # On itère sur les Cons et on vérifie s'ils ont des Pros assignés
        for c in cons:
            covered_pros = [p for p in pros if x[p, c].x > 0.5]
            
            # Si la liste n'est pas vide, c'est que ce Con fait partie de l'explication
            if covered_pros:
                net_value = deltas[c] + sum(deltas[p] for p in covered_pros)
                if print_details:
                    print(f"Groupe dirigé contre {c} ({deltas[c]}) :")
                    print(f"  -> Couvre : {covered_pros} (Poids cumulé Pros: {sum(deltas[p] for p in covered_pros)})")
                    print(f"  -> Bilan net : +{net_value}")
                
        return True

    elif m.status == GRB.INFEASIBLE:
        if print_details:
            print("Résultat : INFEASIBLE (Certificat de non-existence).")
        
        return False
    
for u in data.index:
    for v in data.index:
        if u == v:
            continue
        
        print(f"\nAnalyse (m-1) pour la paire ({u} > {v}) :")
        res = analyse_m_1(data, weights, u, v)
        
        resultats[(u, v)] = resultats.get((u, v), False) or res


Analyse (m-1) pour la paire (Odéon (Ligne 4) > Place d'Italie (Lign 6)) :
Arguments Pour (Pros) : ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Arguments Contre (Cons) : ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
------------------------------
Résultat : Une explication de type (m-1) existe !

Groupe dirigé contre off-peak-entering-passengers/h (-75.24454477050415) :
  -> Couvre : ['peak-entering-passengers/h'] (Poids cumulé Pros: 85.99376545200472)
  -> Bilan net : +10.749220681500574
Groupe dirigé contre Station degradation level ([0,20]  scale) (-94.0556809631303) :
  -> Couvre : ['off-peak-passing-passengers/h'] (Poids cumulé Pros: 96.7429861335053)
  -> Bilan net : +2.687305170374998
Groupe dirigé contre connectivity index [0,100] (-112.86681715575621) :
  -> Couvre : ['strategic priority [0,10]'] (Poids cumulé Pros: 128.9906481780071)
  -> Bilan net : +16.12383102225

# Eplication mixte de type (1-m) ou (m-1)

In [12]:
def analyse_mixte(data, weights, u, v, print_details=True):
    """
    Réalise une analyse mixte (1-m) ou (m-1) de la paire u, v
    
    Combine les deux approches : certains Pros peuvent couvrir plusieurs Cons (1-m)
    et certains Cons peuvent être couverts par plusieurs Pros (m-1)

    Args:
        data (pd.DataFrame): Données des candidats
        weights (dict): Poids des critères
        u (str): Nom du candidat u
        v (str): Nom du candidat v
        print_details (bool): Indique si les détails doivent être affichés
    """
    
    deltas = {}
    for k in weights:
        deltas[k] = weights[k] * (data.loc[u, k] - data.loc[v, k])  

    pros = [k for k, v in deltas.items() if v > 0]
    cons = [k for k, v in deltas.items() if v < 0]

    if print_details:
        print(f"Arguments Pour (Pros) : {pros}")
        print(f"Arguments Contre (Cons) : {cons}")

    # MODÉLISATION GUROBI - APPROCHE MIXTE
    m = Model("Explication_Mixte")

    # Variables de décision MIXTES
    # x[p, c] = 1 si l'argument Pro 'p' couvre l'argument Con 'c'
    # (Flexible : un Pro peut couvrir plusieurs Cons OU un Con peut être couvert par plusieurs Pros)
    x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

    m.update()

    # CONTRAINTES

    # A. Couverture complète : Chaque Con doit être couvert par au moins un Pro
    for c in cons:
        m.addConstr(quicksum(x[p, c] for p in pros) >= 1, name=f"Cover_{c}")

    # B. Validité du Trade-off
    # Formule : Delta_p + Somme(Delta_c * x_pc) >= epsilon
    # Chaque Pro utilisé doit justifier son utilisation
    epsilon = 0.001 

    for p in pros:
        m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")

    # RÉSOLUTION
    m.params.outputflag = 0  
    m.optimize()

    # ANALYSE
    if print_details:
        print("-" * 30)
    if m.status == GRB.OPTIMAL:
        if print_details:
            print("Résultat : Une explication mixte existe !\n")
        
        # Analyse par Pros
        for p in pros:
            covered_cons = [c for c in cons if x[p, c].x > 0.5]
            
            if covered_cons:
                net_value = deltas[p] + sum(deltas[c] for c in covered_cons)
                if print_details:
                    print(f"Groupe mené par {p} (+{deltas[p]}) :")
                    print(f"  -> Couvre : {covered_cons} (Poids cumulé Cons: {sum(deltas[c] for c in covered_cons)})")
                    print(f"  -> Bilan net : +{net_value}")
        
        # Analyse par Cons (pour voir les Cons non "dominés")
        if print_details:
            print()
        for c in cons:
            covering_pros = [p for p in pros if x[p, c].x > 0.5]
            if len(covering_pros) > 1:
                if print_details:
                    print(f"Argument contre {c} ({deltas[c]}) couvert par {len(covering_pros)} Pros : {covering_pros}")
                
        return True

    elif m.status == GRB.INFEASIBLE:
        if print_details:
            print("Résultat : INFEASIBLE (Certificat de non-existence).")
        
        return False

# Exécution de l'analyse mixte
print("\n" + "="*50)
print("ANALYSE MIXTE")
print("="*50)

resultats_mixte = {}
for u in data.index:
    for v in data.index:
        if u == v:
            continue
        
        print(f"\nAnalyse mixte pour la paire ({u} > {v}) :")
        res = analyse_mixte(data, weights, u, v)
        
        resultats_mixte[(u, v)] = res


ANALYSE MIXTE

Analyse mixte pour la paire (Odéon (Ligne 4) > Place d'Italie (Lign 6)) :
Arguments Pour (Pros) : ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Arguments Contre (Cons) : ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
------------------------------
Résultat : Une explication mixte existe !

Groupe mené par peak-entering-passengers/h (+85.99376545200472) :
  -> Couvre : ['off-peak-entering-passengers/h'] (Poids cumulé Cons: -75.24454477050415)
  -> Bilan net : +10.749220681500574
Groupe mené par off-peak-passing-passengers/h (+96.7429861335053) :
  -> Couvre : ['Station degradation level ([0,20]  scale)'] (Poids cumulé Cons: -94.0556809631303)
  -> Bilan net : +2.687305170374998
Groupe mené par strategic priority [0,10] (+128.9906481780071) :
  -> Couvre : ['connectivity index [0,100]'] (Poids cumulé Cons: -112.86681715575621)
  -> Bilan net : +16.123831022250883


# Conclusion : Récupération de la station prioritaire pour être rénovée

Après avoir lancer l'analyse mixte qui est la synthèse des deux analyses précédentes, on obtient naturellement toutes les paires pour lesquelles il est possible d'expliquer qu'une station est prioritaire sur l'autre.

Il est donc possible de comparer les résultats de toutes les paires pour trouver les stations prioritaires, et le cas échéant, afficher une explication.

In [13]:
# Trouveur d'éléments maximaux
def trouver_elements_maximaux(resultats):
    """
    Trouve les éléments maximaux à partir du dictionnaire de résultats.
    
    Un élément u est maximal s'il n'existe pas de v tel que (v, u) soit True
    dans les résultats. Cela signifie que u ne peut pas être dominé par 
    un autre candidat.
    
    Args:
        resultats (dict): Dictionnaire avec clés (u, v) et valeurs booléennes
        
    Returns:
        list: Liste des éléments maximaux
    """
    # Récupérer tous les candidats uniques
    tous_candidats = set()
    for u, v in resultats.keys():
        tous_candidats.add(u)
        tous_candidats.add(v)
    
    # Trouver les candidats qui dominent chaque candidat
    dominants = {candidat: [] for candidat in tous_candidats}
    
    for (u, v), existe_explication in resultats.items():
        if existe_explication:  # Si une explication existe (u > v)
            dominants[v].append(u)  # v est dominé par u
    
    # Les éléments maximaux sont ceux qui n'ont aucun dominant
    elements_maximaux = [candidat for candidat in tous_candidats if not dominants[candidat]]
    
    if print_details:
        print("="*50)
        print("ÉLÉMENTS MAXIMAUX")
        print("="*50)
        if elements_maximaux:
            print(f"Nombre d'éléments maximaux : {len(elements_maximaux)}")
            print(f"Éléments maximaux : {elements_maximaux}")
            print("\nDétails :")
            for candidat in elements_maximaux:
                print(f"  • {candidat} : aucun candidat ne peut le dominer")
        else:
            print("Aucun élément maximal (peut indiquer un cycle ou une relation complète)")
    
    return elements_maximaux

# Exécution
print_details = True
max_elements = trouver_elements_maximaux(resultats)

ÉLÉMENTS MAXIMAUX
Nombre d'éléments maximaux : 3
Éléments maximaux : ['Odéon (Ligne 4)', 'Nation (Ligne 9)', 'Jussieu (Ligne 7)']

Détails :
  • Odéon (Ligne 4) : aucun candidat ne peut le dominer
  • Nation (Ligne 9) : aucun candidat ne peut le dominer
  • Jussieu (Ligne 7) : aucun candidat ne peut le dominer


In [16]:
def explication_element_maximal(element_maximal, data, weights):
    """
    Fournit une explication pour un élément maximal donné.
    
    Args:
        element_maximal (str): Le candidat maximal à expliquer
        data (pd.DataFrame): Données des candidats
        weights (dict): Poids des critères
        resultats (dict): Dictionnaire avec clés (u, v) et valeurs booléennes
    """
    print(f"\nExplications pour l'élément maximal : {element_maximal}")
    for v in data.index:
        if element_maximal == v:
            continue
        
        print(f"\nAnalyse mixte pour la paire ({u} > {v}) :")
        analyse_mixte(data, weights, element_maximal, v, print_details=True)
        
for u in max_elements:
    explication_element_maximal(u, data, weights)


Explications pour l'élément maximal : Odéon (Ligne 4)

Analyse mixte pour la paire (Odéon (Ligne 4) > Place d'Italie (Lign 6)) :
Arguments Pour (Pros) : ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Arguments Contre (Cons) : ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
------------------------------
Résultat : Une explication mixte existe !

Groupe mené par peak-entering-passengers/h (+85.99376545200472) :
  -> Couvre : ['off-peak-entering-passengers/h'] (Poids cumulé Cons: -75.24454477050415)
  -> Bilan net : +10.749220681500574
Groupe mené par off-peak-passing-passengers/h (+96.7429861335053) :
  -> Couvre : ['Station degradation level ([0,20]  scale)'] (Poids cumulé Cons: -94.0556809631303)
  -> Bilan net : +2.687305170374998
Groupe mené par strategic priority [0,10] (+128.9906481780071) :
  -> Couvre : ['connectivity index [0,100]'] (Poids cumulé Cons: -112.8668171557562